# AlexNet Implementation

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf

In [2]:
all_ann = pd.read_csv("lib/datasets/LISA/allAnnotations.csv", delimiter=';')
all_ann

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,"Occluded,On another road",Origin file,Origin frame number,Origin track,Origin track frame number
0,aiua120214-0/frameAnnotations-DataLog02142012_...,stop,862,104,916,158,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2667,stop_1330545910.avi,2
1,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimitUrdbl,425,197,438,213,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2667,stop_1330545910.avi,2
2,aiua120214-0/frameAnnotations-DataLog02142012_...,stop,922,88,982,148,"1,0",aiua120214-0/DataLog02142012_external_camera.avi,2672,stop_1330545910.avi,7
3,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,447,193,461,210,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2672,stop_1330545910.avi,7
4,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,469,189,483,207,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2677,stop_1330545910.avi,12
...,...,...,...,...,...,...,...,...,...,...,...
7850,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,41,209,65,239,"0,0",vid9/MVI_0121.MOV,8813,speedLimit_1324866802.avi,22
7851,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,526,213,543,233,"0,0",vid9/MVI_0121.MOV,8875,speedLimit_1324866807.avi,2
7852,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,546,208,564,230,"0,0",vid9/MVI_0121.MOV,8880,speedLimit_1324866807.avi,7
7853,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,573,204,592,228,"0,0",vid9/MVI_0121.MOV,8885,speedLimit_1324866807.avi,12


In [3]:
all_ann = all_ann.drop_duplicates(subset=["Filename"], keep=False, ignore_index=True)

In [5]:
all_ann = all_ann[all_ann["Occluded,On another road"] != "1,0"]

In [11]:
all_ann.to_csv("lib/datasets/test.csv")

In [8]:
sub_sample = np.load("lib/datasets/sample_negative.npy")
sub_sample

array([ 1236,  2567,  6935,  3568,  6272,  5432,  7058,  6802,  6774,
        6697,   643,  9856,  6516,  6867,  5692, 10785,  1135,   259,
        3021,   808,  9477,  9601,  1102,  4867,  1979,  6322,  8075,
          70, 11194,    23,  1984,  6016,  2811,   991,  1051, 11152,
          15, 11368,  7266,  2064,  9497,  6299,  4519,   474,  5343,
       10260,  8907,  9789,  4011,  7595,  7375,  4643,   790,   714,
       10167,  1746, 10359,  9715,  2623,   506,  4685, 10768,  3940,
        3570,  4413,  8049,  4024,  6599,  1694,  5618,  9918,  1660,
        4158, 11339,  7483,   254,  2166,  3632,  9444,  1026,  3590,
        1494,  1558,  2427,  1632,  5641,  8668,  9668,  6918,   659,
        7848,  5453,  2393,  4592,  6717,  7872,  8815,  2478,  4771,
        6397,  9644, 10861,  8017,  1817,  1630,  6391,  5819,  4483,
        4318, 10115,    95,  1928, 10267,  9326,   117,  4322,  2980,
        2462,  2046,  2783,  3404, 10634, 10276,  6897,  8547,  7796,
        8740,   981,

In [6]:
#max image size
# gt 260 400
height = 227
width = 227
data = []
labels = []
path = "lib/datasets/LISA/"
test = 0
import random
sub_sample = random.sample(list(range(len(all_ann["Filename"]))), k=len(all_ann["Filename"])-2000) # The data set is too big for my PC
for i in sub_sample:
    file = all_ann["Filename"][i]
    sign = all_ann["Annotation tag"][i]
    image = Image.open(path+file)
    image = image.resize((width,height))
    image = np.asarray(image)
    data.append(image)
    labels.append(sign)

In [8]:
# 1k random samples of images with no signs
with open("lib/datasets/LISA/negatives/negatives.dat") as f:
    negatives = [line.rstrip('\n') for line in f]
path_n = "lib/datasets/LISA/negatives/"
import random
sub_sample = random.sample(list(range(len(negatives))), k=1000)
for i in sub_sample:
    file = negatives[i]
    sign = "None"
    image = Image.open(path_n+file)
    image = image.resize((width,height))
    image = np.asarray(image)
    data.append(image)
    labels.append(sign)

In [9]:
signs = np.array(data)
labels = np.array(labels)

# Randomize order
s = np.arange(signs.shape[0])
np.random.seed(43)
np.random.shuffle(s)
signs = signs[s]
labels = labels[s]

In [10]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import  RandomOverSampler
X_train, X_test, y_train, y_test = train_test_split(signs, labels, test_size=0.2, random_state=1)
del signs
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [11]:
# Preprocess
X_train = X_train.astype('float64')
X_val = X_val.astype('float64')
X_test = X_test.astype('float64')

# Normalize
mean_image = np.mean(X_train, axis = 0)
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image

In [12]:
def one_hot(lab, vals):
    hot = []
    for i in range(len(vals)):
        hot.append(lab.tolist().index(vals[i]))
    return hot

In [13]:
# Encode
lab = np.unique(labels)
from tensorflow.keras.utils import to_categorical
y_train_e = np.array(one_hot(lab,y_train))
y_val_e = np.array(one_hot(lab,y_val))

In [14]:
tmp = pd.DataFrame()
tmp["Label"] = lab
tmp["Encoding"] =list(range(len(lab)))
tmp.to_csv("App/lib/datasets/key1.csv",index=False)

### AlexNet

In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, ZeroPadding2D, BatchNormalization

In [1]:
model=Sequential()

model.add(Conv2D(96,(11,11),strides=4,activation='relu',input_shape=X_train.shape[1:],padding='valid'))
model.add(MaxPool2D((3,3),strides=2))
model.add(BatchNormalization())

model.add(ZeroPadding2D(padding=2))
model.add(Conv2D(256,(5,5),activation='relu',padding='same'))
model.add(MaxPool2D((3,3), strides=2))
model.add(BatchNormalization())

model.add(ZeroPadding2D(padding=1))
model.add(Conv2D(384,(3,3),activation='relu',padding='same'))

model.add(ZeroPadding2D(padding=1))
model.add(Conv2D(384,(3,3),activation='relu',padding='same'))

model.add(ZeroPadding2D(padding=1))
model.add(Conv2D(256,(3,3),activation='relu',padding='same'))

model.add(Flatten())
model.add(Dense(4096,activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(4096,activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(len(lab),activation="softmax"))

model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.optimizers.SGD(learning_rate=0.001),metrics=['accuracy'])
epochs = 74
history = model.fit(X_train, y_train_e, batch_size=32, epochs=epochs,
validation_data=(X_val, y_val_e))

NameError: name 'tf' is not defined

In [17]:
model.save("App/lib/models/AlexNet_Negatives")

INFO:tensorflow:Assets written to: App/lib/models/AlexNet_Negatives\assets


In [22]:
y_test_e = one_hot(lab,y_test)
y_pred = np.argmax(model.predict(X_test),axis=1)
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test_e, y_pred))

Accuracy: 0.8716265499635303
